In [1]:
import pandas as pd
import numpy as np
import datetime as dt
import matplotlib
import matplotlib.pyplot as plt
import os
import geopandas as gpd
import seaborn as sns

# suppress warning
import warnings
warnings.filterwarnings('ignore')

import statsmodels.formula.api as smf
import statsmodels.api as sm
from statsmodels.tsa.seasonal import seasonal_decompose
import scipy
from sklearn.metrics import r2_score
from statsmodels.graphics.tsaplots import plot_acf

#import seaborn as sns
#sns.set()

%matplotlib inline



In [2]:
from google.colab import drive
drive.mount("/content/drive",force_remount=True)

Mounted at /content/drive


In [3]:
os.chdir('/content/drive/MyDrive/Data/')
dataDir = '/content/drive/MyDrive/Data/'


## Data Cleaning

In [4]:
# Get the current directory

# os.chdir('/Shared with me/DS24spring-Final-Project/')

# Define the path to the data folder
# data_folder = os.path.join(os.getcwd(), 'Data')

# List all files in the data folder
files = os.listdir()

current_directory = os.getcwd()

# # Define the path to the data folder
data_folder = os.path.join(current_directory, '')

# # List all files in the data folder
files = os.listdir(data_folder)


# Read each CSV file in the data folder
for file in files:
    if file.endswith('listings2024_03.csv'):
        print("df2024_03")
        file_path1 = os.path.join(data_folder, file)
        df2024_03 = pd.read_csv(file_path1)
    elif file.endswith('listings2024_02.csv'):
        print("df2024_02")
        file_path2 = os.path.join(data_folder, file)
        df2024_02 = pd.read_csv(file_path2)
    elif file.endswith("listings2024_01.csv"):
        print("df2024_01")
        file_path3 = os.path.join(data_folder, file)
        df2024_01 = pd.read_csv(file_path3)
    elif file.endswith("listings2023_12.csv"):
        print("df2023_12")
        file_path4 = os.path.join(data_folder, file)
        df2023_12 = pd.read_csv(file_path4)
    elif file.endswith("listings2023_11.csv"):
        print("df2023_11")
        file_path5 = os.path.join(data_folder, file)
        df2023_11 = pd.read_csv(file_path5)
    elif file.endswith("listings2023_10.csv"):
        print("df2023_10")
        file_path6 = os.path.join(data_folder, file)
        df2023_10 = pd.read_csv(file_path6)
    elif file.endswith("listings2023_09.csv"):
        print("df2023_09")
        file_path7 = os.path.join(data_folder, file)
        df2023_09 = pd.read_csv(file_path7)
    elif file.endswith("listings2023_08.csv"):
        print("df2023_08")
        file_path8 = os.path.join(data_folder, file)
        df2023_08 = pd.read_csv(file_path8)
    elif file.endswith("listings2023_07.csv"):
        print("df2023_07")
        file_path9 = os.path.join(data_folder, file)
        df2023_07 = pd.read_csv(file_path9)
    elif file.endswith("listings2023_06.csv"):
        print("df2023_06")
        file_path10 = os.path.join(data_folder, file)
        df2023_06 = pd.read_csv(file_path10)
    elif file.endswith("listings2023_05.csv"):
        print("df2023_05")
        file_path11 = os.path.join(data_folder, file)
        df2023_05 = pd.read_csv(file_path11)
    elif file.endswith("listings2023_04.csv"):
        print("df2023_04")
        file_path12 = os.path.join(data_folder, file)
        df2023_04 = pd.read_csv(file_path12)




df2023_06
df2023_11
df2023_10
df2023_07
df2023_05
df2023_08
df2023_04
df2023_09
df2023_12
df2024_02
df2024_03
df2024_01


In [5]:
dataframes = [
    df2023_08, df2024_01, df2023_09, df2024_02, df2024_03,
    df2023_07, df2023_12, df2023_06, df2023_10, df2023_04,
    df2023_05, df2023_11
]
merged_df = pd.concat(dataframes)

mode_lat_long = merged_df.groupby('id')[['latitude', 'longitude']].apply(lambda x: x.mode().iloc[0]).reset_index()


mean_price = merged_df.groupby('id')['price'].mean().reset_index()


data = pd.merge(mode_lat_long, mean_price, on='id')

data.head()


,id,latitude,longitude,price
0,2595,40.75356,-73.98559,240.000000
1,5121,40.68535,-73.95512,64.363636
2,5136,40.66265,-73.99454,246.666667
3,5178,40.76457,-73.98317,68.000000
4,5203,40.80380,-73.96751,75.000000


In [6]:
import geopandas as gpd
from shapely.geometry import Point

In [7]:
#data.set_index('id', inplace=True)
data['latitude'] = data['latitude'].astype(float)
data['longitude'] = data['longitude'].astype(float)

data['geometry'] = data.apply(lambda x: Point(x['longitude'], x['latitude']), axis=1)


data.dropna(inplace=True)

data.drop(['latitude', 'longitude'], axis=1, inplace=True)

geo_data = gpd.GeoDataFrame(data, geometry='geometry')

In [9]:
geo_data.set_crs("EPSG:4326",inplace=True)

,id,price,geometry
0,2595,240.000000,POINT (-73.98559 40.75356)
1,5121,64.363636,POINT (-73.95512 40.68535)
2,5136,246.666667,POINT (-73.99454 40.66265)
3,5178,68.000000,POINT (-73.98317 40.76457)
4,5203,75.000000,POINT (-73.96751 40.80380)
...,...,...,...
59120,1106510847587344303,261.000000,POINT (-73.98662 40.74770)
59121,1106511007364775922,224.000000,POINT (-73.98662 40.74770)
59122,1106511008237916202,241.000000,POINT (-73.98662 40.74770)
59123,1106541289402959443,250.000000,POINT (-74.00319 40.72189)


In [10]:

os.chdir('/content/drive/MyDrive/Data/')


output_shapefile = 'output_data.shp'


geo_data.to_file(output_shapefile, driver='ESRI Shapefile')